In [34]:
import os # libray to access Operative System, in this case it was only use to access environment variable that contains HF API key
from transformers import AutoTokenizer, AutoModelForCausalLM # transformers contains Tokenizer and LLM (in this case gemma-2b)
import torch # torch enables CUDA (for GPU processing) https://pytorch.org/ you can pip install over here (make sure to check your GPU version first
# using command in cmd
# WARNING: if you dont have a GPU available on your PC or Laptop better run this code in Colab (Code does not work if you don't have GPU)
from huggingface_hub import HfApi # to access hugging face API (contains dataset to train)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # we check GPU availability
if torch.cuda.is_available(): # if True
    print("We can use GPU") # we can use GPU
else:
    print("We don't use GPU") # we cannot use GPU

We can use GPU


# Connect to Hugging Face API

In [35]:
api = HfApi(token=os.environ.get('HF_TOKEN')) # you can get API Token by creating an HF account https://huggingface.co --> Setting --> tokens
# signed-up users have 100 request per hour

In [6]:
model_name = "google/gemma-2b"  # Replace with a model you have access (you need ask for permission to access gemma-2b) 

tokenizer = AutoTokenizer.from_pretrained(model_name, token = os.environ.get('HF_TOKEN')) # load tokenizer using your HF API Key
model = AutoModelForCausalLM.from_pretrained(model_name, token = os.environ.get('HF_TOKEN')) # load model using your HF API key
# is going to take a while to download (model is about 5 GB), once downloaded it would compile faster in future runs
print(model) # Print the model if you want to take a look for 'proj' layers those are the ones that we can adjust using LoRA (Low Rank Adaptation)

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear(in_features=16384, out_features=2048, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): GemmaRMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): GemmaRMSNorm((2048,), eps=1e-

# First Question

In [7]:
input_text = "What should I do on a trip to Europe?"

input_ids = tokenizer(input_text, return_tensors="pt")
outputs = model.generate(**input_ids, max_length=128)
print(tokenizer.decode(outputs[0]))

<bos>What should I do on a trip to Europe?

The answer to this question is not as simple as it seems. There are many different things to see and do in Europe, and it can be difficult to know where to start.

If you’re planning a trip to Europe, here are some tips to help you get started:

1. Decide what you want to see and do.

There are so many amazing places to see and do in Europe, it can be hard to know where to start. Start by deciding what you want to see and do. Do you want to see the Eiffel Tower in Paris? Or


# Second Question

In [8]:
input_text = "Explain the process of photosynthesis in a way that a child could understand"
input_ids = tokenizer(input_text, return_tensors="pt")
print(input_ids)
outputs = model.generate(**input_ids, max_length=128)
print(tokenizer.decode(outputs[0]))

{'input_ids': tensor([[     2,  74198,    573,   2185,    576, 105156,    575,    476,   1703,
            674,    476,   2047,   1538,   3508]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
<bos>Explain the process of photosynthesis in a way that a child could understand.

A 100-W lightbulb is plugged into a standard $120-\mathrm{V}$ (rms) outlet. Find $(a) I_{\text {mas }}$ and $(b) I_{\max }$ when a device like this is operating at the maximum current allowed by its own internal circuitry. (Such a device is often called a light dimmer.)

A 100-turn, 2.0-cm-diameter coil is at rest with its axis vertical. A uniform magnetic field $60^{\circ}$ away


## Step 2 - Motivation for PEFT
- Try to finetune Gemma on Dolly dataset without LoRA
  * Load and visualize the dataset
  * Initiate the trainer
  * Start the training
  * Training with one single GPU my overload memory

In [9]:
from datasets import load_dataset

dataset_name = "databricks/databricks-dolly-15k"
dataset = load_dataset(dataset_name, split="train[0:1000]")

print(f"Instruction is: {dataset[0]['instruction']}")
print(f"Response is: {dataset[0]['response']}")
dataset

Instruction is: When did Virgin Australia start operating?
Response is: Virgin Australia commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route.


Dataset({
    features: ['instruction', 'context', 'response', 'category'],
    num_rows: 1000
})

# Finetune using Parameter Efficient Fine-tuning (PEFT)
- Create your own dataset
  - Visualizing the dataset
  - Cleaning and Preprocessing
  - Uploading to HF hub
- Fine-tune with the dataset

# Do all the imports

In [10]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

# Create own dataset (to optimize training)

In [11]:
from datasets import load_dataset

dataset_name = "databricks/databricks-dolly-15k"
dataset = load_dataset(dataset_name, split="train")

# Check information from dataset

In [12]:
from collections import defaultdict

categries_count = defaultdict(int)
for __, data in enumerate(dataset):
    categries_count[data['category']] += 1
print(categries_count)

defaultdict(<class 'int'>, {'closed_qa': 1773, 'classification': 2136, 'open_qa': 3742, 'information_extraction': 1506, 'brainstorming': 1766, 'general_qa': 2191, 'summarization': 1188, 'creative_writing': 709})


In [13]:
# filter out those that do not have any context
filtered_dataset = []
for __, data in enumerate(dataset):
    if data["context"]:
        continue
    else:
        text = f"Instruction:\n{data['instruction']}\n\nResponse:\n{data['response']}"
        filtered_dataset.append({"text": text})

print(filtered_dataset[0:2])

[{'text': 'Instruction:\nWhich is a species of fish? Tope or Rope\n\nResponse:\nTope'}, {'text': 'Instruction:\nWhy can camels survive for long without water?\n\nResponse:\nCamels use the fat in their humps to keep them filled with energy and hydration for long periods of time.'}]


# Create new splitted file

In [14]:
# convert to json and save the filtered dataset as jsonl file
import jsonlines as jl
with jl.open('dolly-mini-train-learning.jsonl', 'w') as writer:
    writer.write_all(filtered_dataset[0:])

# Load Data from HF API

In [15]:
from datasets import load_dataset

dataset_name = "joselopez1999/data_bricks_train_learning"
dataset = load_dataset(dataset_name, split="train[0:1000]")
dataset

Dataset({
    features: ['text'],
    num_rows: 1000
})

## Define all the parameters
- LoRA parameters
- bitsandbytes parameters
- training arguments / parameters
- Supervised fine-tuning (SFT) parameters

In [16]:
# define some variables - model names
model_name = "google/gemma-2b"
new_model = "gemma-ft"

################################################################################
# LoRA parameters
################################################################################
# LoRA attention dimension
# lora_r = 64
lora_r = 4
# Alpha parameter for LoRA scaling
lora_alpha = 16
# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################
# Activate 4-bit precision base model loading
use_4bit = True
# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"
# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"
# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################
# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"
# Number of training epochs
num_train_epochs = 1
# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False
# Batch size per GPU for training
per_device_train_batch_size = 4
# Batch size per GPU for evaluation
per_device_eval_batch_size = 4
# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1
# Enable gradient checkpointing
gradient_checkpointing = True
# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3
# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4
# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001
# Optimizer to use
optim = "paged_adamw_32bit"
# Learning rate schedule (constant a bit better than cosine)
lr_scheduler_type = "constant"
# Number of training steps (overrides num_train_epochs)
max_steps = -1
# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03
# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True
# Save checkpoint every X updates steps
save_steps = 25
# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################
# Maximum sequence length to use
max_seq_length = 40 # None
# Pack multiple short examples in the same input sequence to increase efficiency
packing = True # False
# Load the entire model on the GPU 0
# device_map = {"": 0}
device_map="auto"

In [17]:
# Load QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit, # Activates 4-bit precision loading
    bnb_4bit_quant_type=bnb_4bit_quant_type, # nf4
    bnb_4bit_compute_dtype=compute_dtype, # float16
    bnb_4bit_use_double_quant=use_nested_quant, # False
)

In [18]:
# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("Setting BF16 to True")
        bf16 = True
    else:
        bf16 = False

In [21]:
# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    token=os.environ.get('HF_TOKEN'),
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          token=os.environ.get('HF_TOKEN'),
                                          trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [22]:
# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj", "up_proj"]
)

In [23]:
# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard",
)
training_arguments

TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=None,
eval_strategy=no,
eval_use_gather_object=False,
evaluation_strategy=None,
fp16=

In [24]:
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    # formatting_func=format_prompts_fn,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

C:\Users\Jose\PycharmProjects\Gemma_with_Lora\.venv\lib\site-packages\huggingface_hub\utils\_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length, packing. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
C:\Users\Jose\PycharmProjects\Gemma_with_Lora\.venv\lib\site-packages\trl\trainer\sft_trainer.py:195: UserWarning: You passed a `packing` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
C:\Users\Jose\PycharmProjects\Gemma_with_Lora\.venv\lib\site-packages\trl\trainer\sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
C:\Users\Jose\PycharmProjects\Gemma_with_Lora\.venv\lib\site-packages\trl\trainer\sft_tra

Generating train split: 0 examples [00:00, ? examples/s]

In [25]:
# Train model
trainer.train()
trainer.model.save_pretrained(new_model)

C:\Users\Jose\PycharmProjects\Gemma_with_Lora\.venv\lib\site-packages\transformers\models\gemma\modeling_gemma.py:540: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Step,Training Loss
25,5.273000
50,3.686700
75,3.152700
100,2.984200
125,2.922600
150,2.908300
175,2.955900
200,2.771700
225,2.840700
250,2.807300


In [26]:
%load_ext tensorboard
%tensorboard --logdir results/runs

# Prompt the newly fine-tuned model
* Load and MERGE the LoRA weights with the model weights
* Run inference with the same prompt we used to test the pre-trained model

In [39]:
input_text = "What are some of the best places to visit in Canada?"

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [40]:
input_ids = tokenizer(input_text, return_tensors="pt")
outputs = model.generate(**input_ids, max_length=128)
print(tokenizer.decode(outputs[0]))

<bos>What are some of the best places to visit in Canada?

Canada is a country that is known for its natural beauty and diverse landscapes. From the bustling cities of Toronto and Vancouver to the rugged wilderness of the Canadian Rockies, there is something for everyone in this vast country. Here are some of the best places to visit in Canada:

1. Toronto: Canada's largest city, Toronto is a vibrant and cosmopolitan city with a rich history and a thriving arts and culture scene. From the bustling downtown core to the leafy neighbourhoods of the west end, there is something for everyone in this city. Must-see attractions include the CN
